In [110]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv


In [111]:
%%time

df = pd.read_csv("/kaggle/input/titanic/train.csv")


CPU times: user 4.02 ms, sys: 1.01 ms, total: 5.03 ms
Wall time: 5.84 ms


In [ ]:
sns.countplot(x="Sex", hue="Survived", data=train_data)

So to conclude from above plot:
* majority of male are NOT survived
* majority of female are survived
* male are greater than female


In [ ]:
sns.heatmap(data=train_data.corr(), annot=True )

Let\`s observe heatmap
* Low correlations are between Survived and [Fare & Pclass]
* Zero correlations are between Survived and [Age, SibSp, PassengerId and Parch]
* Age & Pclass has normal negative correlation which means that Pclass are devived into Age category
* Fare & Pclass also have normolized negative correlation which is probably means relationship
* Parch and SibSp have normal positive correlation value
* PassengerId does not have any high or low correlation with other feature -> so we drop PassenderId


Tose correlation results tell us that Zero correlation features should be eliminated.
However, Age i think must stay for modelling.

In [112]:
sex_bin = {"female": 0,
           "male": 1}
df['Sex'] = df['Sex'].map(sex_bin)


In [114]:
import re 
def get_title(name):
    first = name.split(',')
    second = first[1].split('.')
    return second[0]


In [115]:
df.Name = df.Name.apply(get_title)

In [116]:
df.Name.unique()

array([' Mr', ' Mrs', ' Miss', ' Master', ' Don', ' Rev', ' Dr', ' Mme',
       ' Ms', ' Major', ' Lady', ' Sir', ' Mlle', ' Col', ' Capt',
       ' the Countess', ' Jonkheer'], dtype=object)

We REPLACED NAME TO TITLE:
* 17 UNIQUE TITLES
* TITLES ARE HELPFUL FOR PREDICTING AGE 
* TITLE HELPS TO PREDICT OTHER MISSING VALUES
* NOW I AM ANALYSING TITLES AND OTHER FEATURES


In [81]:
def best_age(name):
    return df.loc[(df.Name == name)].Age.mean()

WHAT IS DIFFERENCE BETWEEN MEAN & MEDIAN ?

In [117]:
for i in df.Name.unique():
    if (df.loc[(df.Name == i)].Age.isnull().sum()) > 0:
        print(i, "&", df.loc[(df.Name == i)].Age.isnull().sum())
        print(best_age(i))

 Mr & 119
32.368090452261306
 Mrs & 17
35.898148148148145
 Miss & 36
21.773972602739725
 Master & 4
4.574166666666667
 Dr & 1
42.0


In [128]:
df.Age=df.Age.fillna(df.Name.apply(best_age))

In [130]:
df.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age              0
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

* Age column has many missing values 
* Cabin column also has many missing values 
* Embarked column has 2 missing values
Now i should replace missing value to reallistic values:


In [131]:
y_train = df.Survived
X_train = df.drop("Survived", axis=1)


In [132]:
X_train = X_train.drop(["Ticket", "Cabin"], axis=1)

In [133]:
X_train = X_train.drop(["PassengerId"], axis=1)

In [134]:
X_train.head()

,Pclass,Name,Sex,Age,SibSp,Parch,Fare,Embarked
0,3,Mr,1,22.0,1,0,7.2500,S
1,1,Mrs,0,38.0,1,0,71.2833,C
2,3,Miss,0,26.0,0,0,7.9250,S
3,1,Mrs,0,35.0,1,0,53.1000,S
4,3,Mr,1,35.0,0,0,8.0500,S


In [ ]:
X_train.columns

In [169]:
X_train.loc[(X_train.Name == " Miss") & (X_train.SibSp == 0) & (X_train.Parch== 0) & (X_train.Embarked == "C")].count()

Pclass      17
Name        17
Sex         17
Age         17
SibSp       17
Parch       17
Fare        17
Embarked    17
dtype: int64

In [164]:
X_train.loc[(X_train.Name == " Mrs") & (X_train.SibSp == 0) & (X_train.Parch== 0) & (X_train.Embarked == "S")]

,Pclass,Name,Sex,Age,SibSp,Parch,Fare,Embarked
15,2,Mrs,0,55.000000,0,0,16.0000,S
66,2,Mrs,0,29.000000,0,0,10.5000,S
161,2,Mrs,0,40.000000,0,0,15.7500,S
190,2,Mrs,0,32.000000,0,0,13.0000,S
327,2,Mrs,0,36.000000,0,0,13.0000,S
399,2,Mrs,0,28.000000,0,0,12.6500,S
415,3,Mrs,0,35.898148,0,0,8.0500,S
483,3,Mrs,0,63.000000,0,0,9.5875,S
516,2,Mrs,0,34.000000,0,0,10.5000,S
706,2,Mrs,0,45.000000,0,0,13.5000,S


In [135]:
X_train.loc[(X_train.Embarked.isnull())]

,Pclass,Name,Sex,Age,SibSp,Parch,Fare,Embarked
61,1,Miss,0,38.0,0,0,80.0,NaN
829,1,Mrs,0,62.0,0,0,80.0,NaN


In [170]:
X_train.Embarked.fillna("S", inplace=True)

EMBARKED 'S' 
I USED SOME FACTORS TO MAKE SURE THAN 'S' SUITS FOR EACH MISSING VALUES

In [171]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Pclass    891 non-null    int64  
 1   Name      891 non-null    object 
 2   Sex       891 non-null    int64  
 3   Age       891 non-null    float64
 4   SibSp     891 non-null    int64  
 5   Parch     891 non-null    int64  
 6   Fare      891 non-null    float64
 7   Embarked  891 non-null    object 
dtypes: float64(2), int64(4), object(2)
memory usage: 55.8+ KB


In [196]:
features = ['Pclass', 'Sex', 'Age' ,'SibSp', 'Parch', 'Fare', 'Embarked']
X = pd.get_dummies(X_train[features], drop_first=True)

In [197]:
X.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked_Q,Embarked_S
0,3,1,22.0,1,0,7.2500,0,1
1,1,0,38.0,1,0,71.2833,0,0
2,3,0,26.0,0,0,7.9250,0,1
3,1,0,35.0,1,0,53.1000,0,1
4,3,1,35.0,0,0,8.0500,0,1


In [198]:
from sklearn.ensemble import RandomForestClassifier 
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.model_selection import cross_val_score

my_pipeline = Pipeline(steps=[('preprocessor', SimpleImputer()),
                              ('model',  RandomForestClassifier(n_estimators=100, max_depth=5, random_state=7))
                             ])







In [202]:
test = pd.read_csv("/kaggle/input/titanic/test.csv")

In [203]:
test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [204]:
my_test = test.drop(["PassengerId","Cabin", "Ticket"],axis=1)

In [205]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  418 non-null    int64  
 1   Pclass       418 non-null    int64  
 2   Name         418 non-null    object 
 3   Sex          418 non-null    object 
 4   Age          332 non-null    float64
 5   SibSp        418 non-null    int64  
 6   Parch        418 non-null    int64  
 7   Ticket       418 non-null    object 
 8   Fare         417 non-null    float64
 9   Cabin        91 non-null     object 
 10  Embarked     418 non-null    object 
dtypes: float64(2), int64(4), object(5)
memory usage: 36.0+ KB


In [206]:
my_test.Name = my_test.Name.apply(get_title)

In [207]:
my_test.Age=my_test.Age.fillna(my_test.Name.apply(best_age))

In [208]:
my_test.Fare.fillna(test.Fare.mean(), inplace=True)

In [209]:
sex_bin = {"female": 0,
           "male": 1}
my_test['Sex'] = my_test['Sex'].map(sex_bin)

In [211]:
my_test2 = my_test.drop("Name", axis=1)

In [212]:
X_test = pd.get_dummies(my_test2, drop_first=True)

In [213]:
X_test.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked_Q,Embarked_S
0,3,1,34.5,0,0,7.8292,1,0
1,3,0,47.0,1,0,7.0000,0,1
2,2,1,62.0,0,0,9.6875,1,0
3,3,1,27.0,0,0,8.6625,0,1
4,3,0,22.0,1,1,12.2875,0,1


In [214]:
X

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked_Q,Embarked_S
0,3,1,22.000000,1,0,7.2500,0,1
1,1,0,38.000000,1,0,71.2833,0,0
2,3,0,26.000000,0,0,7.9250,0,1
3,1,0,35.000000,1,0,53.1000,0,1
4,3,1,35.000000,0,0,8.0500,0,1
...,...,...,...,...,...,...,...,...
886,2,1,27.000000,0,0,13.0000,0,1
887,1,0,19.000000,0,0,30.0000,0,1
888,3,0,21.773973,1,2,23.4500,0,1
889,1,1,26.000000,0,0,30.0000,0,0


Prediction is Done!

Big part of my task was replcaing missing values 
I must update Age and Ebarked coumns again with more clear algorythm of replacing missing values
I used mean() for Age 
for Embarked I chose "S" because it was the most popular 


In [215]:
from sklearn.ensemble import RandomForestClassifier 


In [216]:
model = RandomForestClassifier()


In [217]:
from sklearn.model_selection import GridSearchCV, StratifiedKFold
params = {"n_estimators": np.arange(50,700,50),
         "max_depth": np.arange(1,10)}
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=7)


In [218]:
grid = GridSearchCV(estimator = model,
                   param_grid = params,
                   cv=skf,
                   n_jobs=-1,
                   verbose=1)
grid.fit(X, y_train)

Fitting 5 folds for each of 117 candidates, totalling 585 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  62 tasks      | elapsed:   13.7s
[Parallel(n_jobs=-1)]: Done 212 tasks      | elapsed:   45.6s
[Parallel(n_jobs=-1)]: Done 462 tasks      | elapsed:  1.8min
[Parallel(n_jobs=-1)]: Done 585 out of 585 | elapsed:  2.3min finished


GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=7, shuffle=True),
             estimator=RandomForestClassifier(), n_jobs=-1,
             param_grid={'max_depth': array([1, 2, 3, 4, 5, 6, 7, 8, 9]),
                         'n_estimators': array([ 50, 100, 150, 200, 250, 300, 350, 400, 450, 500, 550, 600, 650])},
             verbose=1)

In [219]:
grid.best_params_

{'max_depth': 9, 'n_estimators': 600}

In [220]:
grid.best_estimator_

RandomForestClassifier(max_depth=9, n_estimators=600)

In [221]:
grid.best_score_

0.832741196409516

In [222]:
preds = grid.predict(X_test)

In [223]:
output = pd.DataFrame({'PassengerId': test.PassengerId, 'Survived': preds})
output.to_csv('submission.csv', index=False)